# Langsmith
Good for Tracing, Evaluating, Promting <br>
[Langsmith document](https://docs.smith.langchain.com/)

In [3]:
# Setting up Environment Variables
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path(r"/Users/masoudkord/Documents/My Projects/RAG-From-Scratch-Langchain")
load_dotenv()

# Langchain
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAING_ENDPOINT'] = os.getenv('LANGCHAING_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
# AvalAI
os.environ['OPENAI_API_BASE'] = os.getenv('OPENAI_API_BASE')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# RAG Layout

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller, simpler steps or subgoals to make it more manageable. Techniques like Chain of Thought (CoT) and Tree of Thoughts (ToT) use structured reasoning to achieve this, while other methods involve prompting large language models (LLMs) or using external planners. It helps enhance problem-solving and planning efficiency.'

# Indexing